# Reboot - SQL Advanced

Tonight, we will use a Blog SQLite database:

In [7]:
!tree

.
├── data
│   ├── blog.sqlite
│   ├── ecommerce.sqlite
│   ├── exploitable_db.sqlite
│   └── students.sqlite
├── exploit.py
├── recap-correction.ipynb
└── recap.ipynb

1 directory, 7 files


## 1. Schema ERD

❓ Open the `data/blog.sqlite` in DBeaver, explore the schema and draw it on [kitt.lewagon.com/db](https://kitt.lewagon.com/db).

_TODO: Double click this cell and **paste** a screenshot of the schema for future reference_.

---
## 2. Most liked posts

Complete the code to get **the 3 most liked posts**:

In [11]:
import sqlite3

conn = sqlite3.connect("data/blog.sqlite")
c = conn.cursor()

# TODO: write the query
query = """
    SELECT posts.title, posts.content, COUNT(posts.id)
    FROM posts
    JOIN likes ON likes.post_id = posts.id
    GROUP BY posts.id
    ORDER BY COUNT(posts.id) DESC
    LIMIT 3
"""

# TODO: Execute the query
c.execute(query)
rows = c.fetchall()

# TODO: Fetch and print the results
rows

[('Half imagine another.',
  'Nice career practice image. Modern son per share painting successful on.',
  84),
 ('Side foot leader popular.',
  'Relate parent run public choice allow. Establish single far Congress impact course offer.',
  82),
 ('Area paper whatever mean.',
  'Space whose often computer. Yard account stuff section write store somebody. Coach none blue skin finish any.',
  81)]

---

### Pretty Print using _pandas_

The readbility of our `print()` statements are not so good.

Next week, we will introduce [pandas](https://pandas.pydata.org/) which will largely improve the UX of our Data Exploration in Notebooks.

Execute the following cell:

In [80]:
import pandas as pd

def execute_and_print(query):
    conn = sqlite3.connect('data/blog.sqlite')
    c = conn.cursor()
    rows = c.execute(query)
    return pd.DataFrame(rows.fetchall(), columns=[header[0] for header in rows.description])

execute_and_print(query)

,first_name,last_name,COUNT(likes.id)
0,Michael,Allen,236
1,Donna,Ramirez,233
2,Barbara,Hurst,227


Then try again the previous `query`, delegating the job of fetching results + displaying them to the `execute_and_print` function and `pandas`:

In [13]:
pd.read_sql_query(query, conn)

,title,content,COUNT(posts.id)
0,Half imagine another.,Nice career practice image. Modern son per sha...,84
1,Side foot leader popular.,Relate parent run public choice allow. Establi...,82
2,Area paper whatever mean.,Space whose often computer. Yard account stuff...,81


---
## 3. Find the three users who 'liked' the most

In [19]:
query = """
    SELECT users.first_name, users.last_name, COUNT(likes.id)
    FROM users
    JOIN likes ON users.id = likes.user_id
    GROUP BY users.id
    ORDER BY COUNT(likes.id) DESC
    LIMIT 3
"""

pd.read_sql_query(query, conn)

,first_name,last_name,COUNT(likes.id)
0,Michael,Allen,236
1,Donna,Ramirez,233
2,Barbara,Hurst,227


---
## 4. Find the most liked author

In [29]:
pd.read_sql_query("""
    SELECT users.first_name, users.last_name, posts.title, COUNT(likes.id) likes_count
    FROM users
    JOIN likes ON posts.id = likes.post_id
    JOIN posts ON users.id = posts.user_id
    GROUP BY users.id
    ORDER BY likes_count DESC
    LIMIT 1
""", conn)

,first_name,last_name,title,likes_count
0,Teresa,Moore,Still relationship rock surface son wait.,647


---
## 5. Who are the authors of the 3 most liked posts?

In [32]:
pd.read_sql_query("""
    SELECT users.first_name, users.last_name, posts.title, COUNT(likes.id) likes_count
    FROM users
    JOIN posts ON users.id = posts.user_id
    JOIN likes ON posts.id = likes.post_id
    GROUP BY posts.id
    ORDER BY likes_count DESC
    LIMIT 3
""", conn)

,first_name,last_name,title,likes_count
0,Melissa,Henry,Half imagine another.,84
1,Cynthia,Raymond,Side foot leader popular.,82
2,Alexander,Cook,Area paper whatever mean.,81


---
## 6. How many people liked at least one post?

In [41]:
pd.read_sql_query("""
    WITH likes_info AS (
    SELECT *, COUNT(likes.id) likes_count
    FROM users
    JOIN likes ON users.id = likes.user_id
    GROUP BY users.id
    HAVING COUNT(likes.id) >= 1
    )
    
    SELECT COUNT(likes_count)
    FROM likes_info
""", conn)

,COUNT(likes_count)
0,49


---
## 7. Compute the cumulative number of likes per day

In [86]:
pd.read_sql_query("""
    SELECT
        COUNT(likes.id),
        likes.created_at,
        SUM(COUNT(likes.id)) OVER (
            ORDER BY likes.created_at
        ) as cumulative_like_per_day
    FROM likes
    GROUP BY likes.created_at
""", conn)

,COUNT(likes.id),created_at,cumulative_like_per_day
0,24,2019-01-01,24
1,34,2019-01-02,58
2,40,2019-01-03,98
3,36,2019-01-04,134
4,27,2019-01-05,161
...,...,...,...
331,35,2019-12-24,9882
332,33,2019-12-25,9915
333,32,2019-12-26,9947
334,23,2019-12-27,9970


---
## 8. (Optional) Who's the biggest fan of each author?

The biggest fan of an author is defined as the user who liked the most the author's posts.
<br><br>
<details>
    <summary>💡 Click for Hint</summary>
    You might need to use <code>WITH</code>
</details>


In [121]:
pd.read_sql_query("""
    SELECT 
        (authors.first_name || ' ' || authors.last_name) AS author_name,
        posts.title,
        likes.user_id AS liker_id,
        COUNT(likes.user_id) likes_count
    FROM users AS authors
    JOIN posts ON authors.id = posts.user_id
    JOIN likes ON posts.id = likes.post_id
    GROUP BY authors.id, likes.user_id
    ORDER BY likes_count DESC
""", conn)

,author_name,title,liker_id,likes_count
0,Teresa Moore,Direction arm practice for.,9,24
1,Cynthia Raymond,Study best whom party indeed those.,45,24
2,James Jones,Policy environment carry offer.,44,24
3,Cynthia Raymond,A cold drop test build word yeah.,15,23
4,Jennifer Mendez,Never specific would every knowledge exactly m...,29,21
...,...,...,...,...
1189,Melissa Henry,Half imagine another.,42,1
1190,Melissa Henry,Half imagine another.,43,1
1191,Melissa Henry,Half imagine another.,44,1
1192,Melissa Henry,Half imagine another.,48,1


In [120]:
pd.read_sql_query("""
    WITH author_likers AS (
        SELECT 
            (authors.first_name || ' ' || authors.last_name) AS author_name,
            posts.title,
            likes.user_id AS liker_id,
            COUNT(likes.user_id) likes_count
        FROM users AS authors
        JOIN posts ON authors.id = posts.user_id
        JOIN likes ON posts.id = likes.post_id
        GROUP BY authors.id, likes.user_id
        ORDER BY likes_count DESC
    )
    SELECT 
        author_likers.author_name,
        author_likers.title,
        (users.first_name || ' ' || users.last_name) AS best_liker_name,
        MAX(author_likers.likes_count) as likes
    FROM author_likers
    JOIN users ON users.id = author_likers.liker_id
    GROUP BY author_likers.author_name
""", conn)

,author_name,title,best_liker_name,likes
0,Alexander Cook,Best detail hair quality out through together.,Jeremiah Ruiz,20
1,Brenda Griffin,Help give in real instead.,Michael Allen,12
2,Brett Johnston,Nothing article stand school cup choice.,Tristan Kelley,19
3,Brittany Miller,Charge pattern soldier.,Barbara Hurst,16
4,Cynthia Raymond,Study best whom party indeed those.,Donald Gray,24
5,David Boyd,Bed focus tell floor special rate husband.,Barbara Hurst,18
6,Donna Smith,Pattern with government discussion worker.,Karen Brown,20
7,Eric Carroll,National writer senior create focus.,James Payne,18
8,Erin Smith,Member way sort hair democratic condition must.,Lisa Norris,7
9,Grace Kerr,Financial less attack with.,Ashley Brooks,19
